In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
import wandb

In [19]:
wandb.init(project="cifar10-training", config={
    "epochs": 10,
    "batch_size": 64,
    "learning_rate": 0.001,
    "dataset": "CIFAR-10",
    "architecture": "ResNet18",
})
config = wandb.config

epoch,▁█
train_loss,█▁
epoch,2
train_loss,0.43964


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # add mps for apple

In [21]:
# Аугментации данных
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Рандомное обрезание
    transforms.RandomHorizontalFlip(),     # Горизонтальное отражение
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # Нормализация
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [22]:
# Загрузка CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
# Используем ResNet18, предобученную на ImageNet
model = models.resnet18(pretrained=True)

# Заменяем последний слой под количество классов CIFAR-10
model.fc = nn.Linear(model.fc.in_features, 10)  # CIFAR-10 имеет 10 классов
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
# оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Шаговое снижение learning rate

In [25]:
def train_model(model, dataloader, optimizer, criterion, scheduler, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        accuracy = 100.0 * correct / total
        avg_loss = total_loss / len(dataloader)
        scheduler.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss, "train_accuracy": accuracy, "lr": scheduler.get_last_lr()[0]})

def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100.0 * correct / total
    avg_loss = total_loss / len(dataloader)
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    wandb.log({"test_loss": avg_loss, "test_accuracy": accuracy})

In [26]:
train_model(model, train_loader, optimizer, criterion, scheduler, config.epochs)
evaluate_model(model, test_loader, criterion)

Epoch [1/10], Loss: 1.1149, Accuracy: 61.90%
Epoch [2/10], Loss: 0.8248, Accuracy: 72.02%
Epoch [3/10], Loss: 0.7180, Accuracy: 75.57%
Epoch [4/10], Loss: 0.6547, Accuracy: 77.89%
Epoch [5/10], Loss: 0.6142, Accuracy: 79.15%
Epoch [6/10], Loss: 0.5013, Accuracy: 82.78%
Epoch [7/10], Loss: 0.4612, Accuracy: 84.08%
Epoch [8/10], Loss: 0.4434, Accuracy: 84.86%
Epoch [9/10], Loss: 0.4204, Accuracy: 85.70%
Epoch [10/10], Loss: 0.4030, Accuracy: 86.14%
Test Loss: 0.4749, Test Accuracy: 83.85%


In [27]:
torch.save(model.state_dict(), "./cifar10_resnet18.pth")

artifact = wandb.Artifact("cifar10_model", type="model")
artifact.add_file("./cifar10_resnet18.pth")
wandb.log_artifact(artifact)

# Завершаем сессию wandb
# wandb.finish()

<Artifact cifar10_model>

обучал на а100 в колабе про, боже храни такую дешёвую подписку на а100

### 1. **Модель: ResNet18**
   - **Почему ResNet18?**  
     ResNet18 — маленькая и производительная модель с остаточными соединениями, которые предотвращают проблему исчезающих градиентов
   - **Предобученная модель:**  
        Использую ImageNet так как она улучшает обучение и качество, потому что есть заранее извлечённые признаки


---

### 2. **Размеры входа:**
   - **Почему 32x32?**  
     Выбор аугментаций и преобразований для модели основывается на сохранении 32 х 32  размера из CIFAR-10

---

### 3. **Гиперпараметры обучения:**
   - **Batch size: 64**
     - так карта легла
   - **Num epochs: 10**
     - CIFAR-10 — небольшой датасет, поэтому 10 достаточно. Когда будем файн-тюнить на своём датасете, то может поменяться
   - **Learning rate: 0.001**
     - Дефолт для оптимизатора Adam, хорошо подходит для предобученных моделей. Если fine-tuning выполняется на новом датасете, скорость обучения можно попробовать уменьшить до \(10^{-4}\) или \(10^{-5}\) для более осторожной подстройки.

---

### 4. **Аугментации (data augmentations):**
   - **RandomHorizontalFlip:**  
     Подходит для CIFAR-10, так как многие изображения (например, собаки, кошки) симметричны и горизонтальная симметрия не изменяет смысл. Аналогично для одежды
   - **RandomCrop (с padding=4):**  
     Имитирует небольшие вариации в расположении объектов на изображении, что повышает устойчивость модели. Можно попробовать увеличить этот параметр

---

### 5. **Loss Function: CrossEntropyLoss**
   - Используется из-за интерпретации предсказаний модели как вероятностей.

---

### 6. **Оптимизатор: Adam**
   - Adam — устойчивый и быстро сходящийся оптимизатор с небольшим кол-во параметров при файнтюнинге

---

### 7. **Scheduler: Cosine Annealing**
   - **Почему?**  
     Этот scheduler плавно уменьшает learning rate, что помогает модели находить оптимальные веса ближе к концу обучения. Warm-up на начальных этапах ускоряет обучение, позволяя модели быстрее сходиться.

---

<!-- ### 8. **Fine-tuning на новом датасете:**
   - **Замена выходного слоя:**  
     Выходной слой ResNet18 заменяется, чтобы адаптировать модель под количество классов нового датасета.
   - **Сохранение предобученных весов:**  
     Замораживание начальных слоев позволяет сохранить общие признаки (например, края и текстуры) и сфокусировать обучение на высокоуровневых признаках нового датасета. -->

---

TODO: fine-tuning на датасете fashion

In [ ]:
# ### load dataset

# new_transform = transforms.Compose([
#     transforms.Resize((32, 32)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# new_dataset = datasets.ImageFolder(root='./new_data', transform=new_transform)
# new_loader = DataLoader(new_dataset, batch_size=batch_size, shuffle=True)

# # Загрузка предобученной модели
# model.fc = nn.Linear(model.fc.in_features, len(new_dataset.classes))
# model = model.to(device)
# model.load_state_dict(torch.load('./models/fashion_meter_cifar10.pth'))

# # Fine-tuning
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train_model(model, new_loader, optimizer, criterion, num_epochs)

# # Сохранение дообученной модели
# torch.save(model.state_dict(), './models/fashion_meter_finetuned.pth')